In [1]:
import sqlalchemy as db

In [ ]:
engine = db.create_engine("sqlite:///resources/meta-db/sdp-metadata.db")
connection = engine.connect()
metadata = db.MetaData()
databases = db.Table("databases", metadata, autoload=True, autoload_with=engine)

In [3]:
print(databases.columns.keys())

['database_id', 'version', 'environment', 'database_name', 'database_owner', 'is_transient', 'comment', 'created', 'last_altered', 'retention_time']


In [ ]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("SELECT * FROM databases")).fetchall()

In [ ]:
import pandas as pd
from datetime import datetime

now = datetime.now()

df = pd.DataFrame(
    {
        "version": ["0.1.0"],
        "environment": ["cert"],
        "database_name": ["SDP"],
        "database_owner": ["US_CERT_DEV_USER"],
        "is_transient": ["FALSE"],
        "comment": ["SDP Database"],
        "created": [now],
        "last_altered": [now],
        "retention_time": [30],
    }
)


In [ ]:
from sqlalchemy.dialects.sqlite import insert


def insert_on_conflict_nothing(table, conn, keys, data_iter):
    # "a" is the primary key in "conflict_table"
    data = [dict(zip(keys, row)) for row in data_iter]
    stmt = (
        insert(table.table)
        .values(data)
        .on_conflict_do_nothing(index_elements=["version", "environment", "database_name"])
    )
    result = conn.execute(stmt)
    return result.rowcount


with engine.connect() as conn:
    df.to_sql(name="databases", con=conn, if_exists="append", method=insert_on_conflict_nothing, index=False)


OperationalError: (sqlite3.OperationalError) ON CONFLICT clause does not match any PRIMARY KEY or UNIQUE constraint
[SQL: INSERT INTO databases (version, environment, database_name, database_owner, is_transient, comment, created, last_altered, retention_time) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?) ON CONFLICT (version, environment, database_name) DO NOTHING]
[parameters: ('0.1.0', 'cert', 'SDP', 'US_CERT_DEV_USER', 'FALSE', 'SDP Database', '2023-09-22 09:59:32.010335', '2023-09-22 09:59:32.010335', 30)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
import pandas as pd
from datetime import datetime

now = datetime.now()

df = pd.DataFrame(
    [
        ("SDP", "US_CERT_DEV_USER", "FALSE", "SDP Database", now, now, 30),
        ("SDP", "US_PROD_SYSADMIN_USER", "FALSE", "SDP Database", now, now, 30),
    ],
    columns=("database_name", "database_owner", "is_transient", "comment", "created", "last_altered", "retention_time"),
)
df.reset_index(inplace=True)

In [ ]:
df.insert(loc=1, column="version", value="0.1.0")

In [ ]:
import pandas as pd
from datetime import datetime

now = datetime.now()

df = pd.DataFrame(
    [
        ("SDP", "US_CERT_DEV_USER", "FALSE", "SDP Database", now, now, 30),
        ("SDP", "US_PROD_SYSADMIN_USER", "FALSE", "SDP Database", now, now, 30),
    ],
    columns=("database_name", "database_owner", "is_transient", "comment", "created", "last_altered", "retention_time"),
)
df.reset_index(inplace=True)
df.insert(loc=1, column="version", value="0.1.0")
df.insert(loc=2, column="environment", value=["cert", "prod"])
df.set_index(["version", "environment", "database_name"], inplace=True)
print(df)


                                   index         database_owner is_transient  \
version environment database_name                                              
0.1.0   cert        SDP                0       US_CERT_DEV_USER        FALSE   
        prod        SDP                1  US_PROD_SYSADMIN_USER        FALSE   

                                        comment                    created  \
version environment database_name                                            
0.1.0   cert        SDP            SDP Database 2023-09-22 14:31:42.397272   
        prod        SDP            SDP Database 2023-09-22 14:31:42.397272   

                                                last_altered  retention_time  
version environment database_name                                             
0.1.0   cert        SDP           2023-09-22 14:31:42.397272              30  
        prod        SDP           2023-09-22 14:31:42.397272              30  


In [ ]:
with engine.connect() as conn:
    df.to_sql(name="databases", con=conn, if_exists="replace")


In [ ]:
df = pd.DataFrame(
    [
        ("SDP", "US_DEV_DEV_USER", "FALSE", "SDP Database", now, now, 30),
        ("SDP", "US_NON_PROD_DEV_USER", "FALSE", "SDP Database", now, now, 30),
    ],
    columns=("database_name", "database_owner", "is_transient", "comment", "created", "last_altered", "retention_time"),
)
df.reset_index(inplace=True)
df.insert(loc=1, column="version", value="0.1.0")
df.insert(loc=2, column="environment", value=["dev", "non_prod"])
df.set_index(["version", "environment", "database_name"], inplace=True)
with engine.connect() as conn:
    df.to_sql(name="databases", con=conn, if_exists="append")

In [ ]:
import json
from schema_sentinel.metadata_manager.model.database import Database

db = Database(
    version="0.1.0",
    environment="dev",
    database_name="SDP",
    database_owner="US_DEV_DEV_USER",
    is_transient="NO",
    comment=None,
    created="2023-04-01",
    last_altered="2023-09-01",
    retention_time=30,
)
db.database_id = db.__get_id__()


def get_database_id(database: Database) -> str:
    return database.database_id


def get_schema_id(database: Database, schema_name: str) -> str:
    id = json.loads(database.database_id)
    id["schema_name"] = schema_name
    return json.dumps(id)


db
get_schema_id(db, "AUDIT")


'{"database_name": "SDP", "version": "0.1.0", "environment": "dev", "schema_name": "AUDIT"}'

In [ ]:
import json
from schema_sentinel.metadata_manager.model.database import Database

left = Database(
    version="0.1.0",
    environment="dev",
    database_name="SDP",
    database_owner="US_DEV_DEV_USER",
    is_transient="NO",
    comment=None,
    created="2023-04-01",
    last_altered="2023-09-01",
    retention_time=30,
)
left.database_id = left.__get_id__()

right = Database(
    version="0.1.3",
    environment="dev",
    database_name="SDP",
    database_owner="US_DEV_DEV_USER",
    is_transient="NO",
    comment=None,
    created="2023-04-01",
    last_altered="2023-09-20",
    retention_time=30,
)
right.database_id = right.__get_id__()

In [23]:
left.__get_df__()

,Attribute,Value
0,version,0.1.0
1,environment,dev
2,database_name,SDP
3,database_owner,US_DEV_DEV_USER
4,is_transient,NO
5,comment,None
6,created,2023-04-01
7,last_altered,2023-09-01
8,retention_time,30
9,database_id,"{""database_name"": ""SDP"", ""version"": ""0.1.0"", ""..."


In [14]:
right.__get_df__()

,Attribute,Value
0,version,0.1.3
1,environment,dev
2,database_name,SDP
3,database_owner,US_DEV_DEV_USER
4,is_transient,NO
5,comment,None
6,created,2023-04-01
7,last_altered,2023-09-20
8,retention_time,30
9,database_id,"{""database_name"": ""SDP"", ""version"": ""0.1.3"", ""..."


In [15]:
left.__side_by_side__(right)

,Attribute,Left,Right
0,version,0.1.0,0.1.3
1,environment,dev,dev
2,database_name,SDP,SDP
3,database_owner,US_DEV_DEV_USER,US_DEV_DEV_USER
4,is_transient,NO,NO
5,comment,None,None
6,created,2023-04-01,2023-04-01
7,last_altered,2023-09-01,2023-09-20
8,retention_time,30,30
9,database_id,"{""database_name"": ""SDP"", ""version"": ""0.1.0"", ""...","{""database_name"": ""SDP"", ""version"": ""0.1.3"", ""..."


In [16]:
Database.__to_df__([left, right])

,version,environment,database_name,database_owner,is_transient,comment,created,last_altered,retention_time,database_id
0,0.1.0,dev,SDP,US_DEV_DEV_USER,NO,None,2023-04-01,2023-09-01,30,"{""database_name"": ""SDP"", ""version"": ""0.1.0"", ""..."
0,0.1.3,dev,SDP,US_DEV_DEV_USER,NO,None,2023-04-01,2023-09-20,30,"{""database_name"": ""SDP"", ""version"": ""0.1.3"", ""..."


In [ ]:
import json
from schema_sentinel.metadata_manager.model.comparison import Comparison

comparison = Comparison(
    object_type="column",
    comparison_key="MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT [SDP:0.1.3->0.1.1]",
    source_database_id=json.dumps({"database_name": "SDP", "version": "0.1.3", "environment": "dev"}),
    target_database_id=json.dumps({"database_name": "SDP", "version": "0.1.1", "environment": "dev"}),
    comparison_value=json.dumps(
        {
            "key": "MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT [SDP:0.1.3->0.1.1]",
            "comparison": {
                "left": "Column",
                "right": "Column",
                "differences": {
                    "ordinal_position": [18, 2],
                    "is_nullable": ["YES", "NO"],
                    "character_maximum_length": [10, 16777216],
                    "character_octet_length": [40, 16777216],
                    "column_default": ["NULL", "'DEV'"],
                    "comment": ["NULL", "One of DEV, NONPROD, CERT or PROD"],
                },
            },
        }
    ),
    comparison_performed_by="user@example.com",
    created="2023-09-20 13:17:59.920765",
)
comparisons = [comparison]
comparison = Comparison(
    object_type="column",
    comparison_key="MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT [SDP:0.1.1->0.1.3]",
    target_database_id=json.dumps({"database_name": "SDP", "version": "0.1.3", "environment": "dev"}),
    source_database_id=json.dumps({"database_name": "SDP", "version": "0.1.1", "environment": "dev"}),
    comparison_value=json.dumps(
        {
            "key": "MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT [SDP:0.1.1->0.1.3]",
            "comparison": {
                "left": "Column",
                "right": "Column",
                "differences": {
                    "ordinal_position": [2, 18],
                    "is_nullable": ["NO", "YES"],
                    "character_maximum_length": [16777216, 10],
                    "character_octet_length": [16777216, 40],
                    "column_default": ["'DEV'", "NULL"],
                    "comment": ["One of DEV, NONPROD, CERT or PROD", "NULL"],
                },
            },
        }
    ),
    comparison_performed_by="user@example.com",
    created="2023-09-20 13:17:59.920765",
)
comparison.diffs


{"key": "MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT [SDP:0.1.1->0.1.3]", "comparison": {"left": "Column", "right": "Column", "differences": {"ordinal_position": [2, 18], "is_nullable": ["NO", "YES"], "character_maximum_length": [16777216, 10], "character_octet_length": [16777216, 40], "column_default": ["'DEV'", "NULL"], "comment": ["One of DEV, NONPROD, CERT or PROD", "NULL"]}}}


,DB Key,Left Object,Right Object,Object Name,Attribute,Left,Right
0,[SDP:0.1.1->0.1.3],Column,Column,[MIGRATIONS.SCHEMA_DISCREPANCY.ENVIRONMENT],ordinal_position,2,18
1,,,,,is_nullable,NO,YES
2,,,,,character_maximum_length,16777216,10
3,,,,,character_octet_length,16777216,40
4,,,,,column_default,'DEV',NULL
5,,,,,comment,"One of DEV, NONPROD, CERT or PROD",NULL


In [ ]:
import json
from schema_sentinel.metadata_manager.model.comparison import Comparison

comparison = Comparison(
    object_type="column_constraint",
    comparison_key="CORE.CUSTOMER_ACCOUNT.UNIVERSE.FK_DARE_DEPOSIT_SUCCESS_CUSTOMER_ACCOUNT_ID [SDP:0.1.3->0.1.1]",
    source_database_id=json.dumps({"database_name": "SDP", "version": "0.1.3", "environment": "dev"}),
    target_database_id=json.dumps({"database_name": "SDP", "version": "0.1.1", "environment": "dev"}),
    comparison_value=json.dumps(
        {
            "key": "CORE.CUSTOMER_ACCOUNT.UNIVERSE.FK_DARE_DEPOSIT_SUCCESS_CUSTOMER_ACCOUNT_ID [SDP:0.1.3->0.1.1]",
            "comparison": {"left": "ColumnConstraint"},
        }
    ),
    comparison_performed_by="user@example.com",
    created="2023-09-20 13:17:59.920765",
)
comparison.diffs

{"key": "CORE.CUSTOMER_ACCOUNT.UNIVERSE.FK_DARE_DEPOSIT_SUCCESS_CUSTOMER_ACCOUNT_ID [SDP:0.1.3->0.1.1]", "comparison": {"left": "ColumnConstraint"}}


,DB Key,Left Object,Right Object,Object Name,Attribute,Left,Right
0,[SDP:0.1.3->0.1.1],ColumnConstraint,Not present,[CORE.CUSTOMER_ACCOUNT.UNIVERSE.FK_DARE_DEPOSI...,,,
